In [10]:
# REMINDER: USE ENVIRONMENT "alkemy"
%reset -s -f
import logging
import os

import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import pyro

In [11]:
smoke_test = ('CI' in os.environ)
assert pyro.__version__.startswith('1.8.3')

pyro.enable_validation(True)
pyro.set_rng_seed(1)
logging.basicConfig(format='%(message)s', level=logging.INFO)

# Set matplotlib settings
%matplotlib inline
plt.style.use('default')

In [12]:
import pyro.distributions as dist
print("10 Bernoulli trials:")
print( dist.Bernoulli(0.7).sample() )
print( dist.Bernoulli(0.7).sample() )
print( dist.Bernoulli(0.7).sample() )
print( dist.Bernoulli(0.7).sample() )
print( dist.Bernoulli(0.7).sample() )
print( dist.Bernoulli(0.7).sample() )
print( dist.Bernoulli(0.7).sample() )
print( dist.Bernoulli(0.7).sample() )
print( dist.Bernoulli(0.7).sample() )
print( dist.Bernoulli(0.7).sample() )

print("\nBinomial(10,0.7):")
print( dist.Binomial(10,0.7).sample() )

10 Bernoulli trials:
tensor(0.)
tensor(1.)
tensor(1.)
tensor(0.)
tensor(1.)
tensor(0.)
tensor(1.)
tensor(0.)
tensor(1.)
tensor(1.)

Binomial(10,0.7):
tensor(8.)


In [13]:
list(map( lambda x: pyro.sample("actualSample",x), # Sample from
    list(map(lambda x: dist.Bernoulli(x), [0.7]*10 )) # <{ 0 3/10, 1 7/10 }, { 0 3/10, 1 7/10 }, { 0 3/10, 1 7/10 }, { 0 3/10, 1 7/10 }, { 0 3/10, 1 7/10 }, { 0 3/10, 1 7/10 }, { 0 3/10, 1 7/10 }, { 0 3/10, 1 7/10 }, { 0 3/10, 1 7/10 }, { 0 3/10, 1 7/10 }>
))

[tensor(1.),
 tensor(1.),
 tensor(0.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.),
 tensor(1.)]

In [14]:
n = 100

# Let's say that the reference distribution is precisely
# <{ 0 3/10, 1 7/10 }, { 0 3/10, 1 7/10 }, { 0 3/10, 1 7/10 }, { 0 3/10, 1 7/10 }, { 0 3/10, 1 7/10 }, { 0 3/10, 1 7/10 }, { 0 3/10, 1 7/10 }, { 0 3/10, 1 7/10 }, { 0 3/10, 1 7/10 }, { 0 3/10, 1 7/10 }>
class_0_ref = 3/10
class_1_ref = 7/10
threshold = 0.1

# Black Box
output_black_box = list( map(lambda x: pyro.sample("actualSample",x), [dist.Bernoulli(8/10)]*n) ) # output_black_box = callClassifier(20000,2,2,1,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0)
class_0_freq = 1 - sum(output_black_box)/len(output_black_box)
class_1_freq = sum(output_black_box)/len(output_black_box)
print("Black box: \n", output_black_box)
print( "Class 0:", class_0_freq )
print( "Class 1:", class_1_freq )

distance = max( abs(class_0_freq - class_0_ref), abs(class_1_freq - class_1_ref) )
print("Distance between frequency and reference:", distance)
print("Does the black box approximate the reference distribution?", distance<=threshold )

Black box: 
 [tensor(1.), tensor(1.), tensor(1.), tensor(0.), tensor(1.), tensor(0.), tensor(0.), tensor(1.), tensor(1.), tensor(1.), tensor(0.), tensor(1.), tensor(1.), tensor(1.), tensor(0.), tensor(1.), tensor(0.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(0.), tensor(0.), tensor(1.), tensor(1.), tensor(0.), tensor(1.), tensor(1.), tensor(0.), tensor(0.), tensor(0.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(0.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(0.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(1.), tensor(0.), tensor(1.), te

In [74]:
n = 2000

df = pd.read_excel('defaults.xls', sheet_name='Data')
#print(df.columns) # Print column names
#df.head()

filtered_df_1 = df[(df['X2'] == 1)] # SEX = Male and MARITAL = Married
#print(filtered_df_1)
print("Criterio 1: n=", len(filtered_df_1))

filtered_df_2 = df[(df['X2'] == 2)] # SEX = Female and MARITAL = Married
#print(filtered_df_2)
print("Criterio 2: n=", len(filtered_df_2))

print("Proportion of male subjects:",len(filtered_df_1)/len(df), "\nProportion of female subjects:", len(filtered_df_2)/len(df))
if (abs(len(filtered_df_1)/len(df) - len(filtered_df_2)/len(df))>0.1):
    print("WARNING: Classes are imbalanced\n")

random_rows_1 = filtered_df_1.sample(n) #possibly subsample
random_rows_1_tensor = tensor_array = [torch.tensor(val) for val in random_rows_1['Y']]
#print(random_rows_1_tensor)

random_rows_2 = filtered_df_2.sample(n) #possibly subsample
random_rows_2_tensor = tensor_array = [torch.tensor(val) for val in random_rows_2['Y']]
#print(random_rows_2_tensor)

# MALE case
male_0_ref = 75/100
male_1_ref = 25/100
threshold = 0.05

# Black Box frequencies
male_0_freq = 1 - sum(random_rows_1_tensor)/len(random_rows_1_tensor)
male_1_freq = sum(random_rows_1_tensor)/len(random_rows_1_tensor)
print( "Male 0 freq:", male_0_freq )
print( "Male 1 freq:", male_1_freq )

distance = max( abs(male_0_freq - male_0_ref), abs(male_1_freq - male_1_ref) )
print("Distance between frequency and reference:", distance)
print("Does the black box approximate the reference distribution?", distance<=threshold )

# Set reference
female_0_ref = 75/100
female_1_ref = 25/100

# Black Box frequencies
female_0_freq = 1 - sum(random_rows_2_tensor)/len(random_rows_2_tensor)
female_1_freq = sum(random_rows_2_tensor)/len(random_rows_2_tensor)
print( "\nFemale 0 freq:", female_0_freq )
print( "Female 1 freq:", female_1_freq )

distance = max( abs(female_0_freq - female_0_ref), abs(female_1_freq - female_1_ref) )
print("Distance between frequency and reference:", distance)
print("Does the black box approximate the reference distribution?", distance<=threshold )

print("\nNow let's look at the difference between frequencies:")
distance = max( abs(female_0_freq - male_0_freq), abs(female_1_freq - male_1_freq) )
print("Distance between frequency and reference:", distance)
print("Does the black box approximate the reference distribution?", distance<=threshold )

Criterio 1: n= 11888
Criterio 2: n= 18112
Proportion of male subjects: 0.3962534582180594 
Proportion of female subjects: 0.6037132095596813

Male 0 freq: tensor(0.7555)
Male 1 freq: tensor(0.2445)
Distance between frequency and reference: tensor(0.0055)
Does the black box approximate the reference distribution? tensor(True)

Female 0 freq: tensor(0.7825)
Female 1 freq: tensor(0.2175)
Distance between frequency and reference: tensor(0.0325)
Does the black box approximate the reference distribution? tensor(True)

Now let's look at the difference between frequencies:
Distance between frequency and reference: tensor(0.0270)
Does the black box approximate the reference distribution? tensor(True)
